In [59]:
from jaxtyping import Shaped, jaxtyped
import numpy as np
import torch
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, set_seed
set_seed(42)
from tinygrad import nn
from tinygrad.tensor import Tensor
Tensor.manual_seed(42)
from tinygrad.dtype import dtypes
from tinygrad import TinyJit
from typechecker import typechecker

from nanogpt.model import Block, TinyConfig, GPT2

%matplotlib inline

model = GPT2.load_from_huggingface()
# model(Tensor.ones((4, TinyConfig.sequence_length), dtype=dtypes.int)).numpy()

ram used:  1.03 GB, h.1.ln_1.bias                                     :  11%| | 

wte.weight (50257, 768)
wpe.weight (1024, 768)
h.0.ln_1.weight (768,)
h.0.ln_1.bias (768,)
h.0.attn.c_attn.weight (2304, 768)
h.0.attn.c_attn.bias (2304,)
h.0.attn.c_proj.weight (768, 768)
h.0.attn.c_proj.bias (768,)
h.0.ln_2.weight (768,)
h.0.ln_2.bias (768,)
h.0.mlp.c_fc.weight (3072, 768)
h.0.mlp.c_fc.bias (3072,)
h.0.mlp.c_proj.weight (768, 3072)
h.0.mlp.c_proj.bias (768,)
h.1.ln_1.weight (768,)
h.1.ln_1.bias (768,)
h.1.attn.c_attn.weight (2304, 768)
h.1.attn.c_attn.bias (2304,)
h.1.attn.c_proj.weight (768, 768)
h.1.attn.c_proj.bias (768,)
h.1.ln_2.weight (768,)
h.1.ln_2.bias (768,)
h.1.mlp.c_fc.weight (3072, 768)
h.1.mlp.c_fc.bias (3072,)
h.1.mlp.c_proj.weight (768, 3072)
h.1.mlp.c_proj.bias (768,)
h.2.ln_1.weight (768,)
h.2.ln_1.bias (768,)
h.2.attn.c_attn.weight (2304, 768)
h.2.attn.c_attn.bias (2304,)
h.2.attn.c_proj.weight (768, 768)
h.2.attn.c_proj.bias (768,)
h.2.ln_2.weight (768,)
h.2.ln_2.bias (768,)
h.2.mlp.c_fc.weight (3072, 768)
h.2.mlp.c_fc.bias (3072,)
h.2.mlp.c_proj.

ram used:  1.03 GB, lm_head.weight                                    : 100%|█| 


loaded weights in 1390.22 ms, 0.00 GB loaded at 0.00 GB/s


In [62]:
import tiktoken

num_return_sequences = 5
max_length = 30

enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
x = Tensor(tokens, dtype="long", device="metal").unsqueeze(0).repeat(num_return_sequences, 1)

while x.size(1) < max_length:
    logits = model(x)
    assert logits.shape == (*x.shape, 50257)
    logits = logits[:, -1, :]
    assert logits.shape == (x.shape[0], 50257)
    probs = logits.softmax(axis=-1)
    assert probs.shape == (x.shape[0], 50257)

    ix = probs.multinomial()
    assert ix.shape == (x.shape[0], 1)
    x = Tensor.cat(x, ix, dim=1)



# while x.size(1) < max_length:



# encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
# decode = lambda l: enc.decode(l)

# start_ids = encode("Hello, I'm a language model,")
# idx = (Tensor(start_ids)[None, ...])
# max_new_tokens = 16
# temp = 1.0
# top_k = 40

# for _ in range(max_new_tokens):
#     idx_cond = idx 
#     logits = model(idx_cond)
#     logits = logits[:, -1, :] / temp
#     idx_next = logits.softmax().multinomial()
#     idx = Tensor.cat(idx, idx_next, dim=1)

# decode(idx.tolist()[0])

for i in range(num_return_sequences):
    tokens = x.numpy()[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> Hello, I'm a language model, not a app. I've never got lazy. No matter what manuals you have on your audience adaptable if
> Hello, I'm a language model, that is how I met Yamatanagi." We sat down for lunch. As we left Cologne June 15th
> Hello, I'm a language model, much like I use desktop editors. But this time, people writing on modern languages are having the whole process,
> Hello, I'm a language model, and Brian Conway's Immortal Language, if you so please let me know. I'd love this story to eventually
> Hello, I'm a language model, I'm writing about philosophy and self-concepts. When you write an award winning book of philosophy and there


In [56]:
enc.decode(x.numpy()[0])

"Hello, I'm a language model, and all this talk does……although it seems like there are lots of crazy special cases lurking nearby. Maybe there"

In [27]:
model(x).numpy().shape

(5, 8, 50257)

In [20]:
x.size()

8

In [63]:
import tiktoken
import torch.nn.functional as F

num_return_sequences = 5
max_length = 30

enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).repeat(num_return_sequences, 1)

model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124 M
model_ = model_hf.to('mps')
x = tokens.to("mps")

while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model_(x)[0] # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> Hello, I'm a language model, which means I'm familiar with it, but I'm not fluent in that. Well, with that said,
> Hello, I'm a language model, and the syntax, to make use of it, is pretty good. So why do you have that and not
> Hello, I'm a language model, I'm doing this work in Python, and then I'm writing code for Haskell.

So we can
> Hello, I'm a language model, and you're making assumptions about my use of them. I'm not a natural language learner. I'm
> Hello, I'm a language model, well, I'm from Java and have to write a programming language for it. I have my own vocabulary because


In [3]:
from tinygrad.nn.state import get_state_dict

get_state_dict(model)

{'wte.weight': <Tensor <UOp METAL (50257, 768) float ShapeTracker(views=(View(shape=(50257, 768), strides=(768, 1), offset=0, mask=None, contiguous=True),))> on METAL with grad None>,
 'wte.arange': <Tensor <UOp METAL (50257, 1) int ShapeTracker(views=(View(shape=(50257, 1), strides=(1, 0), offset=0, mask=None, contiguous=True),))> on METAL with grad None>,
 'wpe.weight': <Tensor <UOp METAL (1024, 768) float ShapeTracker(views=(View(shape=(1024, 768), strides=(768, 1), offset=0, mask=None, contiguous=True),))> on METAL with grad None>,
 'wpe.arange': <Tensor <UOp METAL (1024, 1) int ShapeTracker(views=(View(shape=(1024, 1), strides=(1, 0), offset=0, mask=None, contiguous=True),))> on METAL with grad None>,
 'h.0.ln_1.weight': <Tensor <UOp METAL (768,) float (<Ops.BUFFER: 11>, <buf real:True device:METAL size:768 dtype:dtypes.float>)> on METAL with grad None>,
 'h.0.ln_1.bias': <Tensor <UOp METAL (768,) float (<Ops.BUFFER: 11>, <buf real:True device:METAL size:768 dtype:dtypes.float>)> 

In [4]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124 M
sd_hf = model_hf.state_dict()
for k, v in sd_hf.items():
    print(f"'{k}': {v.shape}, {type(v)}")

'transformer.wte.weight': torch.Size([50257, 768]), <class 'torch.Tensor'>
'transformer.wpe.weight': torch.Size([1024, 768]), <class 'torch.Tensor'>
'transformer.h.0.ln_1.weight': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_1.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_attn.weight': torch.Size([768, 2304]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_attn.bias': torch.Size([2304]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_proj.weight': torch.Size([768, 768]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_proj.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_2.weight': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_2.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_fc.weight': torch.Size([768, 3072]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_fc.bias': torch.Size([3072]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_proj.weight': torch.Size([3072, 768]), <class 'torch.

In [5]:
prompt = "The quick brown fox"

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tok = tokenizer(prompt, return_tensors="pt")
input_ids = tok["input_ids"]
model_hf = model_hf.eval()
with torch.no_grad():
    outputs = model_hf(input_ids=input_ids)
    logits_hf = outputs.logits

In [ ]:
# model = GPT2.load_from_huggingface()
# for k, v in model.state_dict().items():
#     print(f"'{k}': {v.shape}, {type(v)}")

'transformer.wte.weight': torch.Size([50257, 768]), <class 'torch.Tensor'>
'transformer.wpe.weight': torch.Size([1024, 768]), <class 'torch.Tensor'>
'transformer.h.0.ln_1.weight': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_1.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_attn.weight': torch.Size([2304, 768]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_attn.bias': torch.Size([2304]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_proj.weight': torch.Size([768, 768]), <class 'torch.Tensor'>
'transformer.h.0.attn.c_proj.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_2.weight': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.ln_2.bias': torch.Size([768]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_fc.weight': torch.Size([3072, 768]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_fc.bias': torch.Size([3072]), <class 'torch.Tensor'>
'transformer.h.0.mlp.c_proj.weight': torch.Size([768, 3072]), <class 'torch.

In [8]:
# model.eval()
# with torch.no_grad():
logits = model(tokens=Tensor(input_ids.numpy())).realize().numpy()
logits

array([[[-36.287136, -35.011105, -38.07908 , ..., -40.516106,
         -41.37571 , -34.919064],
        [-62.31387 , -61.56446 , -66.49383 , ..., -68.128494,
         -68.32271 , -63.58289 ],
        [-66.32398 , -66.74522 , -72.16191 , ..., -75.19552 ,
         -73.46512 , -68.17866 ],
        [-88.29095 , -88.723694, -93.442245, ..., -98.621155,
         -90.63791 , -90.99138 ]]], shape=(1, 4, 50257), dtype=float32)

In [19]:
from tinygrad import Device

Device.DEFAULT

'METAL'

In [4]:
import tiktoken
import torch.nn.functional as F

num_return_sequences = 5
max_length = 30

enc = tiktoken.get_encoding("gpt2")
start_ids = enc.encode("Hello, I'm a language model,")
x = (Tensor(start_ids)[None, ...])
idx = x

# for _ in range(max_length):
#     idx_cond = idx
#     logits = model(idx_cond)
#     logits = logits[:, -1, :] / 1
#     idx_next = logits.softmax().multinomial()
#     idx = Tensor.cat(idx, idx_next, dim=1)

# decoded = enc.decode(idx.tolist())
# print(decoded)

tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).repeat(num_return_sequences, 1)
tokens = Tensor(tokens.numpy(), requires_grad=False)

# model_ = model_hf.to('mps')

# tokens = tokens.to("mps")
while tokens.size(1) < max_length:
    # forward the model to get the logits
    # with torch.no_grad():
    logits = model(tokens).realize().numpy() # (B, T, vocab_size)
    # take the logits at the last position
    logits = logits[:, -1, :] # (B, vocab_size)
    logits = torch.from_numpy(logits)
    # get the probabilities
    probs = F.softmax(logits, dim=-1)
    # do top-k sampling of 50 (huggingface pipeline default)
    # topk_probs here becomes (5, 50), topk_indices is (5, 50)
    topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
    # select a token from the top-k probabilities
    # note: multinomial does not demand the input to sum to 1
    ix = torch.multinomial(topk_probs, 1) # (B, 1)
    # gather the corresponding indices
    xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
    # append to the sequence
    tokens = torch.cat((torch.from_numpy(tokens.numpy()), xcol), dim=1)
    tokens = Tensor(tokens.numpy(), requires_grad=False)
    print('hi')

# print the generated text
for i in range(num_return_sequences):
    tokens = tokens[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

TypeError: argument 'tokens': 'list' object cannot be interpreted as an integer

In [33]:
res = idx.realize()
res

<Tensor <UOp METAL (1, 38) int ShapeTracker(views=(View(shape=(1, 38), strides=(0, 1), offset=0, mask=None, contiguous=True),))> on METAL with grad None>

In [37]:
idx.numpy()

array([[15496,    11,   314,  1101,   257,  3303,  2746,    11,   543,
        12716,   617,   661,  4327,   284,   892,   318,   517,   546,
          366,   270,   338,   691,   351,  3037,   290,   407,  9061,
         1600,   340,   338,  4414,  2723,    13,   198,   198,    40,
        12546,    11]], dtype=int32)

In [42]:
enc.decode(idx.numpy()[0])

'Hello, I\'m a language model, which unfortunately some people tend to think is more about "it\'s only with technology and not computers", it\'s benefit source.\n\nI disagree,'

In [38]:
decoded = enc.decode(idx.numpy())

TypeError: argument 'tokens': only integer scalar arrays can be converted to a scalar index

> Hello, I'm a language model, a programming language. I have to read the code and I have to do the thing where they come together.
> Hello, I'm a language model, I am able to model the semantics of types. All that requires is that the programmer know what values there are
> Hello, I'm a language model, not a model of writing. When I started writing Ruby on Rails, all my work was focused on Rails and
> Hello, I'm a language model, a language model where languages are constantly at various points in their lives. My friends and I tend to develop language
> Hello, I'm a language model, so there isn't anything about it that's fundamentally wrong... that's what I'm doing. But I feel


In [9]:
np.allclose(logits.numpy(), logits_hf.numpy(), rtol=1e-4, atol=1e-6)

True